### Install and import necessary packages

In [ ]:
BUCKET_NAME = "YOUR_BUCKET"
# Change the bucket name with your own

In [ ]:
%%capture
!pip install -r /home/neuro/codes/questionnaire_brain/requirements_multivariate_prediction.txt;

In [ ]:
import json
import os
import sys
import boto3
import botocore
import os
import numpy as np
import pandas as pd
import nibabel as nib
# Required for regular expression matching
import re  
# Reqiured for removing files once they are stored in s3 bucket
import shutil 
from glmsingle.glmsingle import GLM_single
# Prevent the usage of global variables
from noglobal import NoGlobal
noglobal = NoGlobal(globals()).noglobal
# Prevent warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Prints the Python version and saves package requirements
print(sys.version)
!pip freeze > requirements_my_glmsingle_social_isolation.txt

In [ ]:
# Here, we decide your working directory
initial_directory = "/home/neuro/mount/"
data_directory = os.path.join(initial_directory, "run_glmsingle_social_isolation_data/")
# Create data_directory if it doesn't exist
if not os.path.exists(data_directory):
    os.makedirs(data_directory)
print(data_directory)

### Set up AWS s3

In [ ]:
# Configures AWS directories and copies credentials.
if not os.path.exists('/root/.aws'):
    os.makedirs('/root/.aws')
    # Put credential on host into guest
    ! cp /home/neuro/credential/your_s3_credentials ~/.aws/credentials

In [ ]:
# Function to download and upload files to/from AWS S3.
@noglobal
def cp_s3(BUCKET_NAME, FROM_PATH, TO_PATH, FILE_NAME, download_or_upload):
    s3_resource = boto3.resource('s3')
    bucket = s3_resource.Bucket(BUCKET_NAME)

    from_path = os.path.join(FROM_PATH, FILE_NAME)
    to_path = os.path.join(TO_PATH, FILE_NAME)
    
    if download_or_upload == "download":
        bucket.download_file(from_path, to_path)
        print("Downloading from s3")
    elif download_or_upload == "upload":
        bucket.upload_file(from_path, to_path)
        print("Uploading to s3")
    else:
        raise ValueError('Please specify download or upload.')
    return

### Download Data

In [ ]:
# Load RepetitionTime (preferrably do this only for the first time and than erase them with #)
session_id = "sub-SAXSISO01b"
filename = "/ds003242-fmriprep/" + session_id + "/func/" + session_id + "_task-CIC_run-1_space-MNI152NLin6Asym_desc-smoothAROMAnonaggr_bold.json"
s3link = "s3://openneuro-derivatives/fmriprep" + filename
tr_download_dir = os.path.join(data_directory,"TR", "tr.json")

In [ ]:
!aws s3 cp --no-sign-request $s3link $tr_download_dir

json_open = open(tr_download_dir, 'r')
json_load = json.load(json_open)
RepetitionTime = json_load['RepetitionTime']

In [ ]:
# Downloads neuroimaging data from the same session from OpenNeuro
# Note that some session_id have missing files.
# To accommodate this, we first check for runs that have both bold and event file and download only those to avoid error.
def download_openneuro_func(session_id):
    # Initialize AWS S3 client
    s3 = boto3.client('s3', config= boto3.session.Config(signature_version=botocore.UNSIGNED))

    # List of available files in the dataset
    bold_bucket_name = 'openneuro-derivatives'
    event_bucket_name = 'openneuro.org'
    bold_prefix = f'fmriprep/ds003242-fmriprep/{session_id}/func/'
    event_prefix = f'ds003242/{session_id}/func/'

    data_files = []
    event_files = []
    

    # Fetch the list of available bold and event files from each S3 bucket
    available_bold_files = s3.list_objects(Bucket=bold_bucket_name, Prefix=bold_prefix)
    available_event_files = s3.list_objects(Bucket=event_bucket_name, Prefix=event_prefix)
    
    
    for file_info in available_bold_files.get('Contents', []):
        file_name = file_info['Key']
        
        # Remove 'fmriprep/ds003242-fmriprep/' from the local download directory to set up directory to save bold and event files
        # local_file_name = session_id/func/filename.bold.nii.gz
        local_file_name = file_name.replace('fmriprep/ds003242-fmriprep/', '')
        download_dir = os.path.join(data_directory,local_file_name)

        # Check if the file matches the pattern for data files
        if "_space-MNI152NLin6Asym_desc-smoothAROMAnonaggr_bold.nii.gz" in file_name and "task-CIC" in file_name:
            
            print(f"Download_dir: {download_dir}")
    
            s3link = f"s3://{bold_bucket_name}/{file_name}"
            run_number = re.search(r"run-(\d+)_", file_name).group(1)
    
            print(f"Downloading data file of {session_id} run {run_number}")
            if not os.path.exists(download_dir):
                !aws s3 cp --no-sign-request $s3link $download_dir
            else:
                print(f"Data file of {session_id} run {run_number} already exists")
            data_files.append(download_dir)

            # Create the corresponding event file name
            event_file_name = f"ds003242/{session_id}/func/{session_id}_task-CIC_run-00{run_number}_events.tsv"
            event_s3link = f"s3://{event_bucket_name}/{event_file_name}"
            event_download_dir = os.path.join(data_directory, session_id, "func", f"{session_id}_task-CIC_run-00{run_number}_events.tsv")
            
            print(f"Event_downlaod_dir: {event_download_dir}")
            
            try:
                print(f"Downloading event file of {session_id} run {run_number}")
                !aws s3 cp --no-sign-request $event_s3link $event_download_dir
            except Exception as e:
                raise RuntimeError(f"Event file {event_file_name} does not exist")
        
            event_files.append(event_download_dir)

    print("Downloading all done!")
    return data_files, event_files

### Define functions to conduct GLMsingle

In [ ]:
# Performs Generalized Linear Model analysis using the downloaded data by GLMsingle.
def glmsingle_func(in_files, eventfiles, RepetitionTime):
    # get affine transformation matrices for all bold files
    print("Getting affine data...")
    affine_list = []
    for num in range(len(in_files)):
        nii1=nib.load(in_files[num])
        affine = nii1.header.get_best_affine()
        affine_list.append(affine)
    
    print("Comparing affine data across runs...")    
    # If the contents of the affine_list differ between runs, throw an error.
    for num in range(len(affine_list)-1):
        if not np.allclose(affine_list[num], affine_list[num+1]):
            raise ValueError('the contents of the affine_list differ between runs')
    
    print("Loading nibabel images to a list...")
    # Load nibabel images to a python list
    data = []
    for d in range(len(in_files)):
        tmp = nib.load(in_files[d])
        data.append(tmp.get_fdata())
        print(f"data shape: {tmp.shape}")
    print(f"data number: {len(data)}")

    tr          = RepetitionTime
    stimdur     = 18
    
    # Set design
    print("Setting design...")
    design = []

    # Because GLMsingle do not allow to set a varying degree of stimdur for different trial types, we do not include events related to "Rating"- it has different stimdur=5
    conds = np.array(['Food', 'Social', 'Control'])

    nconds = conds.shape[0]

    run_events_list = []
    for e in (range(len(eventfiles))):
        run_events = pd.read_csv(eventfiles[e],delimiter = '\t')

        # remove _1, _2, _3
        run_events['trial_type'] = run_events['trial_type'].str[:-2]

        # sec to volume
        run_events["onset_tr"] = run_events["onset"]/tr
        run_events["onset_volume"] = run_events["onset_tr"].round().astype(int)
        run_events = run_events.reset_index(drop=True)
        run_events_list.append(run_events)

        run_design = np.zeros((np.shape(data[e])[3], nconds))
        
        for c, cond in enumerate(conds):
                condidx = np.argwhere(run_events['trial_type'].values == cond)
                condvols = run_events['onset_volume'].values[condidx]
                run_design[condvols, c] = 1
        design.append(run_design)
        print(f"run_design shape: {run_design.shape}")
        
    print(f"design number: {len(design)}")
    
    # Set outputdir
    current_dir = os.getcwd()
    outputdir_glmsingle = os.path.join(current_dir, "GLMestimatesingletrialoutputs")

    # Set opt
    opt = dict()
    glmsingle_obj = GLM_single(opt)
    
    print(f'running GLMsingle...')
    
    if not os.path.exists(outputdir_glmsingle):
    
        # run GLMsingle
        results_glmsingle = glmsingle_obj.fit(
           design,
           data,
           stimdur,
           tr,
           outputdir=outputdir_glmsingle)
    else:
        print(f'loading existing GLMsingle outputs from directory:\n\t{outputdir_glmsingle}')
    
        # load existing file outputs if they exist
        results_glmsingle['typed'] = np.load(os.path.join(outputdir_glmsingle,'TYPED_FITHRF_GLMDENOISE_RR.npy'),allow_pickle=True)

    beta_img = nib.Nifti1Image(results_glmsingle['typed']['betasmd'], affine_list[0])
    beta_img_savepath = os.path.join(os.getcwd(),'beta_img.nii.gz')
    nib.save(beta_img, beta_img_savepath)

    events_savepath = os.path.join(current_dir,'events.csv')
    df_v = pd.concat(run_events_list)
    df_v.to_csv(events_savepath)

    beta_img = beta_img_savepath
    events = events_savepath
        
    return beta_img, events

In [ ]:
# Function to run the analysis for a specific session for bulk analysis later on for all sessions.
def run_analysis(session_id):
    in_files, eventfiles = download_openneuro_func(session_id)

    sub_dir = os.path.join(data_directory, session_id)
    # If you're running this in a Jupyter Notebook and it causes error,
    # you might want to replace the next line with a cell magic command.
    # Otherwise, the os.chdir() function will work in a regular Python script.
    # Set your working directory to yourcurrentworkingdirectory/run_glmsingle_social_isolation_data/session_id
    os.chdir(sub_dir)
    

    beta_img, events = glmsingle_func(in_files, eventfiles, RepetitionTime)

    # Saves the output files back to AWS S3.
    FROM_PATH   = sub_dir
    TO_PATH     = os.path.join("outputdir/glmsingle_output/" + session_id)

    FILE_NAME   = "beta_img.nii.gz"
    cp_s3(BUCKET_NAME, FROM_PATH, TO_PATH, FILE_NAME, "upload")

    FILE_NAME   = "events.csv"
    cp_s3(BUCKET_NAME, FROM_PATH, TO_PATH, FILE_NAME, "upload")

### Actually conduct GLMsingle

In [ ]:
# Checks the disk usage before the operations.
!df -h -m --total

In [ ]:
# Iterate through all session ids and subsets. There are 32 participants with each having three sessions.
skip_numbers = {5, 6, 7, 16, 20, 23, 25, 29, 31, 37} # We skip the numbers with missing files.
for i in range(1, 43): # 1 to 42 inclusive
    if i in skip_numbers:
        continue
    for subset in ['b', 'f', 's']:
        session_id = f"sub-SAXSISO{str(i).zfill(2)}{subset}"
        run_analysis (session_id)

# Finally after conducting GLMsingle for all available data, erase the repetition time file.
shutil.rmtree(data_directory)